# 利用rating2inter.ipynb中U/I的index对features进行一一对应(meta-text)
- Reindex item feature ID with IDs generated in 0rating2inter.ipynb

In [1]:
import os
import pandas as pd

In [2]:
os.chdir('/home/xin/MMRec/Sports14')
os.getcwd()

'/home/xin/XMMRec/Sports14'

In [3]:
# load item mapping
i_id_mapping = 'i_id_mapping.csv'
df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (18357, 2)


,asin,itemID
0,1881509818,0
1,2094869245,1
2,7245456259,2
3,7245456313,3


In [4]:

import gzip, json
meta_file = 'meta_Sports_and_Outdoors.json.gz'

print('0 Extracting U-I interactions.')

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF(meta_file)

print(f'Total records: {meta_df.shape}')
meta_df[:3]

0 Extracting U-I interactions.
Total records: (532197, 9)


,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,0000032069,Adult Ballet Tutu Cheetah Pink,7.89,http://ecx.images-amazon.com/images/I/51EzU6qu...,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi,"[[Sports & Outdoors, Other Sports, Dance, Clot...",NaN,NaN
1,0000031909,Girls Ballet Tutu Neon Pink,7.00,http://ecx.images-amazon.com/images/I/41xBoP0F...,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown,"[[Sports & Outdoors, Other Sports, Dance]]",{'Toys & Games': 201847},High quality 3 layer ballet tutu. 12 inches in...
2,0000032034,Adult Ballet Tutu Yellow,7.87,http://ecx.images-amazon.com/images/I/21GNUNIa...,"{'also_bought': ['B00D2JSRFQ', '0000032042', '...",BubuBibi,"[[Sports & Outdoors, Other Sports, Dance, Clot...",NaN,NaN


In [5]:
# remapping
map_dict = dict(zip(df['asin'], df['itemID']))

meta_df['itemID'] = meta_df['asin'].map(map_dict)
meta_df.dropna(subset=['itemID'], inplace=True)
meta_df['itemID'] = meta_df['itemID'].astype('int64')
#meta_df['description'] = meta_df['description'].fillna(" ")
meta_df.sort_values(by=['itemID'], inplace=True)

print(f'shape: {meta_df.shape}')
meta_df[:2]

shape: (18357, 10)


,asin,title,price,imUrl,related,brand,categories,salesRank,description,itemID
132,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,http://ecx.images-amazon.com/images/I/21iMxsyD...,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"[[Sports & Outdoors, Hunting & Fishing, Huntin...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...,0
155,2094869245,5 LED Bicycle Rear Tail Red Bike Torch Laser B...,8.26,http://ecx.images-amazon.com/images/I/51RtwnJw...,"{'also_bought': ['B0081O93N2', 'B00EYTCHJA', '...",,"[[Sports & Outdoors, Cycling, Lights & Reflect...",{'Sports &amp; Outdoors': 14293},This newly-designed Laser tail light can emit ...,1


In [6]:
ori_cols = meta_df.columns.tolist()

ret_cols = [ori_cols[-1]] + ori_cols[:-1]
print(f'new column names: {ret_cols}')

new column names: ['itemID', 'asin', 'title', 'price', 'imUrl', 'related', 'brand', 'categories', 'salesRank', 'description']


In [7]:
meta_df[:3]

,asin,title,price,imUrl,related,brand,categories,salesRank,description,itemID
132,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,http://ecx.images-amazon.com/images/I/21iMxsyD...,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"[[Sports & Outdoors, Hunting & Fishing, Huntin...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...,0
155,2094869245,5 LED Bicycle Rear Tail Red Bike Torch Laser B...,8.26,http://ecx.images-amazon.com/images/I/51RtwnJw...,"{'also_bought': ['B0081O93N2', 'B00EYTCHJA', '...",,"[[Sports & Outdoors, Cycling, Lights & Reflect...",{'Sports &amp; Outdoors': 14293},This newly-designed Laser tail light can emit ...,1
201,7245456259,Black Mountain Products Single Resistance Band...,10.49,http://ecx.images-amazon.com/images/I/411Ikpf1...,"{'also_bought': ['B00DDBS2JE', 'B00H1KNHE8', '...",Black Mountain,"[[Sports & Outdoors, Exercise & Fitness, Acces...",{'Sports &amp; Outdoors': 1010},Black Mountain Products single resistance band...,2


In [8]:
ret_df = meta_df[ret_cols]
# dump
ret_df.to_csv(os.path.join('./', 'meta-sports14.csv'), index=False)
print('done!')

done!


## Reload

In [9]:
indexed_df = pd.read_csv('meta-sports14.csv')
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

shape: (18357, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,0,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,http://ecx.images-amazon.com/images/I/21iMxsyD...,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"[['Sports & Outdoors', 'Hunting & Fishing', 'H...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...
1,1,2094869245,5 LED Bicycle Rear Tail Red Bike Torch Laser B...,8.26,http://ecx.images-amazon.com/images/I/51RtwnJw...,"{'also_bought': ['B0081O93N2', 'B00EYTCHJA', '...",NaN,"[['Sports & Outdoors', 'Cycling', 'Lights & Re...",{'Sports &amp; Outdoors': 14293},This newly-designed Laser tail light can emit ...
2,2,7245456259,Black Mountain Products Single Resistance Band...,10.49,http://ecx.images-amazon.com/images/I/411Ikpf1...,"{'also_bought': ['B00DDBS2JE', 'B00H1KNHE8', '...",Black Mountain,"[['Sports & Outdoors', 'Exercise & Fitness', '...",{'Sports &amp; Outdoors': 1010},Black Mountain Products single resistance band...
3,3,7245456313,Black Mountain Products Resistance Band Set wi...,32.99,http://ecx.images-amazon.com/images/I/51FdHlZS...,"{'also_bought': ['1612431712', 'B00GSBMW2Y', '...",Black Mountain,"[['Sports & Outdoors', 'Exercise & Fitness', '...",{'Sports &amp; Outdoors': 15},[if gte mso 9]><xml> <o:OfficeDocumentSettings...


In [10]:
## Reload

i_uni = indexed_df['itemID'].unique()

print(f'# of unique items: {len(i_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(i_uni), max(i_uni)))

# of unique items: 18357
min/max of unique learners: 0/18356
